In [ ]:
import librosa
import numpy as np
from numpy.fft import rfft
from numpy import pi
from matplotlib import pyplot as plt
from IPython.display import Audio
import cmath
import scipy
from cache_no_hash import cache
from blindDescend import blindDescend
from yin import yin
from harmonicSynth import HarmonicSynth, Harmonic

TWO_PI = np.pi * 2

In [ ]:
SR = 22050
NYQUIST = SR // 2
PAGE_LEN = 1024
DTYPE = np.float32
N_HARMONICS = 8

In [ ]:
HANN = scipy.signal.get_window('hann', PAGE_LEN, True)
IMAGINARY_LADDER = np.linspace(0, TWO_PI * 1j, PAGE_LEN)

In [ ]:
def sino(freq, length):
    return np.sin(np.arange(length) * freq * TWO_PI / SR)

def playHard(data):
    return Audio(data, rate = SR)
def play(data, soft = .02):
    t = np.concatenate([data, [1]])
    length = round(soft * SR)
    t[:length ] = np.multiply(t[:length ], np.linspace(0, 1, length))
    t[-length:] = np.multiply(t[-length:], np.linspace(1, 0, length))
    return playHard(t)

def findPeaks(energy):
    slope = np.sign(energy[1:] - energy[:-1])
    extrema = slope[1:] - slope[:-1]
    return np.argpartition(
        (extrema == -2) * energy[1:-1], - N_HARMONICS,
    )[- N_HARMONICS:] + 1

def sft(signal, freq_bin):
    # Slow Fourier Transform
    return np.abs(np.sum(signal * np.exp(IMAGINARY_LADDER * freq_bin))) / PAGE_LEN

def refineGuess(guess, signal):
    def loss(x):
        if x < 0:
            return 0
        return - sft(signal, x)
    freq_bin, loss = blindDescend(loss, .01, .4, guess)
    return freq_bin * SR / PAGE_LEN, - loss

def widePlot(h = 3, w = 12):
    plt.gcf().set_size_inches(w, h)

    
def spectro(signal, do_wide = True, trim = 130):
    energy = np.abs(rfft(signal * HANN))
    plt.plot(energy[:trim])
    if do_wide:
        widePlot()

def concatSynth(synth, harmonics, n):
    buffer = []
    for i in range(n):
        synth.eat(harmonics)
        buffer.append(synth.mix())
    return np.concatenate(buffer)

def pitch2freq(pitch):
    return np.exp((pitch + 36.37631656229591) * 0.0577622650466621)

def freq2pitch(f):
    return np.log(f) * 17.312340490667562 - 36.37631656229591

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
raw, SR = librosa.load('range.wav')
raw = raw[30000:-10000]
raw = raw[:(raw.size // PAGE_LEN) * PAGE_LEN]
print(SR)
play(raw)

In [ ]:
a = raw[PAGE_LEN*31:PAGE_LEN*32] * HANN
spec = np.log(np.abs(rfft(a)))
plt.plot(spec)

In [ ]:
f0 = yin(a, SR, PAGE_LEN)
tops = np.array([sft(a, f0 * i / SR * PAGE_LEN) * PAGE_LEN for i in range(20)])
f = interp1d([f0 * i for i in range(20)], tops)
freqs = np.linspace(0, SR/2, int(PAGE_LEN/2)+1)
plt.plot(freqs, np.exp(spec))
plt.plot(freqs[:300], [f(x) for x in freqs[:300]])
[plt.axvline(f0 * i) for i in range(20)]
# plt.axis([0, 6000, -14, 5])
widePlot()
plt.yscale('log')

In [ ]:
f0 = yin(a, SR, PAGE_LEN)
tops = np.log([sft(a, f0 * i / SR * PAGE_LEN) * PAGE_LEN for i in range(20)])
f = interp1d([f0 * i for i in range(20)], tops)
freqs = np.linspace(0, SR/2, int(PAGE_LEN/2)+1)
plt.plot(freqs, np.exp(spec))
plt.plot(freqs[:300], np.exp([f(x) for x in freqs[:300]]))
[plt.axvline(f0 * i) for i in range(20)]
widePlot()

In [ ]:
freqs = np.linspace(0, SR/2, int(PAGE_LEN/2)+1)
plt.plot(freqs, spec)
[plt.axvline(f0 * i) for i in range(20)]
widePlot()

In [ ]:
def getEnvelope(signal, len_signal):
    f0 = yin(signal, SR, len_signal)
    harmonics_f = np.arange(0, NYQUIST, f0)
    harmonics_a = [sft(signal, f_bin) for f_bin in harmonics_f / SR * len_signal]
    harmonics_a[0] = harmonics_a[1]
    f = interp1d(harmonics_f, harmonics_a)
    def envelope(x):
        try:
            return f(x)
        except ValueError:
            return 0
    return envelope

In [ ]:
def video(width = 100):
    cursor = 0
    X = []
    Y = []
    C = []
    y = 0
    while cursor + PAGE_LEN < raw.size:
        signal = raw[cursor : cursor + PAGE_LEN]
        cursor += PAGE_LEN
        y += 1
        envelope = getEnvelope(signal, PAGE_LEN)
        for x in range(0, 6000, 30):
            X.append(x)
            Y.append(y)
            t = (np.log(envelope(x)) + 8) / (3 + 8)
            t = min(1, max(0, t))
            C.append(np.array([1-t, 1-t, t]))
    plt.scatter(x = X, y = Y, c = C, s = 2)
    widePlot(6, 10)
video()

## reconstruction

In [ ]:
def reconstruct(n_harmos = 8):
    hs = HarmonicSynth(
        n_harmos, SR, PAGE_LEN, DTYPE, STUPID_MATCH = True, 
        DO_SWIPE = True, CROSSFADE_RATIO = .3, 
    )
    cursor = 0
    buffer = []
    while cursor + PAGE_LEN < raw.size:
        signal = raw[cursor : cursor + PAGE_LEN]
        cursor += PAGE_LEN
        envelope = getEnvelope(signal, PAGE_LEN)
        f0 = yin(signal, SR, PAGE_LEN)
        harmonics_f = np.arange(1, n_harmos + 1) * f0
        harmonics = [Harmonic(f, envelope(f)) for f in harmonics_f]
        hs.eat(harmonics)
        buffer.append(hs.mix())
    return np.concatenate(buffer)

In [ ]:
play(raw)

In [ ]:
play(reconstruct(8))

In [ ]:
play(reconstruct(80))

Conclusion: 8 is enough

## pitch transfer

In [ ]:
def pitchTransfer(freq = 440, n_harmos = 8):
    hs = HarmonicSynth(
        n_harmos, SR, PAGE_LEN, DTYPE, STUPID_MATCH = True, 
        DO_SWIPE = True, CROSSFADE_RATIO = .3, 
    )
    cursor = 0
    buffer = []
    while cursor + PAGE_LEN < raw.size:
        signal = raw[cursor : cursor + PAGE_LEN]
        cursor += PAGE_LEN
        envelope = getEnvelope(signal, PAGE_LEN)
        f0 = freq
        harmonics_f = np.arange(1, n_harmos + 1) * f0
        harmonics = [Harmonic(f, envelope(f)) for f in harmonics_f]
        hs.eat(harmonics)
        buffer.append(hs.mix())
    return np.concatenate(buffer)

In [ ]:
play(pitchTransfer(330))

## is that you

In [ ]:
raw, SR = librosa.load('isthatyou.wav')
raw = raw[25000:-10000]
raw = raw[:(raw.size // PAGE_LEN) * PAGE_LEN]
print(SR)
play(raw)

In [ ]:
play(reconstruct(60))

In [ ]:
play(pitchTransfer(220, 60))

Conclusion: PAGE_LEN = 256; N_HARMONICS = 60

## white noise for unvoiced

In [ ]:
from scipy.stats import norm

def whitenoise(size):
    return norm.rvs(0, 1, size)
def unvoiced():
    spectrum = whitenoise(PAGE_LEN//2 + 1)
#     spectrum = np.multiply(spectrum, scipy.signal.get_window('hann', PAGE_LEN//2+1, False))
    return np.fft.irfft(spectrum)
play(np.concatenate([unvoiced() for _ in range(20)]))


In [ ]:
def voiced():
    spectrum = np.zeros((PAGE_LEN//2 + 1, ))
    spectrum[round(220 / SR * PAGE_LEN)] = 1
    spectrum[round(440 / SR * PAGE_LEN)] = .8
    spectrum[round(660 / SR * PAGE_LEN)] = .95
#     spectrum = np.multiply(spectrum, scipy.signal.get_window('hann', PAGE_LEN//2+1, False))
    return np.fft.irfft(spectrum)
play(np.concatenate([voiced() for _ in range(20)]))


In [ ]:
from collections import namedtuple

Harmonic = namedtuple('Harmonic', ['freq', 'mag', 'phase'])
TWO_PI_J = 2j * np.pi

def ifftSynth(harmonics, SR, PAGE_LEN):
    spectrum = np.zeros((PAGE_LEN // 2 + 1, ), dtype=np.complex)
    phases = []
    for freq, mag, phase in harmonics:
        try:
            freq_bin = round(freq / SR * PAGE_LEN)
            spectrum[freq_bin] += mag * np.exp(phase * TWO_PI_J)
        except IndexError:
            phases.append(0)
            continue
        else:
            phases.append((phase + PAGE_LEN / SR * (freq_bin * SR / PAGE_LEN)) % 1)
    return np.fft.irfft(spectrum) * PAGE_LEN, phases


In [ ]:
phases = [0, 0, 0]
buffer = []
for i in range(30):
    signal, phases = ifftSynth([
        Harmonic(310, 1, 0), 
#         Harmonic(400, .7, 0), 
#         Harmonic(600, .9, 0), 
    ], SR, PAGE_LEN)
    buffer.append(signal)
play(np.concatenate(buffer))

Damn. All mid-bin freqs divide the page perfectly! no need to phase

In [ ]:
for t in np.linspace(0, 1, 7):
    spec = np.abs(rfft(sino(195 + t * 21, PAGE_LEN)))[6:16]
    print(np.sum(spec ** 2))
    plt.plot(spec ** 2, c=np.array([(1-t)**.5, t**.5, (1-t)**.5]))
widePlot(5, 10)